In [1]:
from copy import deepcopy
from pathlib import Path
import cv2
import matplotlib.pylab as plt
import numpy as np
from scipy.io import loadmat
from scipy.ndimage import gaussian_filter
from tqdm import tqdm
import cv2
import pandas as pd
import numpy as np 
import cv2 
from matplotlib import pyplot as plt 
import math


In [2]:
# Read the bounding box coordinates from the CSV file
csv_path_1 = '129_1_clean.txt'
df1_left_view = pd.read_csv(csv_path_1)

# Read the bounding box coordinates from the CSV file
csv_path_2 = '129_2_clean.txt'
df2_right_view = pd.read_csv(csv_path_2)

In [3]:
display(df1_left_view)
display(df2_right_view)

,frame,id,x,y,x_offset,y_offset
0,1,1,1212.98,416.74,26.04,14.37
1,1,2,1151.14,415.00,27.58,10.34
2,1,3,1487.32,431.94,17.38,19.86
3,1,4,1080.11,337.38,18.68,6.46
4,1,5,1220.90,325.30,21.28,10.54
...,...,...,...,...,...,...
30196,3117,5,1011.60,235.99,10.38,22.01
30197,3117,6,1304.99,217.40,15.01,24.80
30198,3117,7,969.60,489.40,23.90,16.40
30199,3117,8,1037.36,245.93,14.89,29.79


,frame,id,x,y,x_offset,y_offset
0,1,1,1010.39,357.55,21.50,7.74
1,1,2,1151.46,337.71,20.11,17.63
2,1,3,1173.87,341.47,11.20,12.02
3,1,4,1446.30,460.60,19.11,20.90
4,1,5,1066.87,626.29,48.07,6.87
...,...,...,...,...,...,...
30644,3118,5,949.07,696.08,23.25,33.30
30645,3118,6,1067.44,288.15,38.96,21.85
30646,3118,7,1188.06,193.33,33.94,17.14
30647,3118,8,883.91,514.11,25.29,14.69


In [5]:
CameraDataFile = "stereoParams_Dep4.mat"
view1 = 'right'
ImagePath1 = 'frame_800_without_boxes_left.jpg'
view2 = 'left'
ImagePath2 = 'frame_800_without_boxes_right.jpg'

In [6]:
def Undistorter(ImagePath,view,CameraDataFile):
    
    dd = loadmat(CameraDataFile)
    distCoeffs1 = deepcopy(dd["distortionCoefficients1"])
    distCoeffs2 = deepcopy(dd["distortionCoefficients2"])
    cameraMatrix1 = deepcopy(dd["intrinsicMatrix1"])
    cameraMatrix2 = deepcopy(dd["intrinsicMatrix2"])
    R = deepcopy(dd["rotationOfCamera2"])
    T = deepcopy(dd["translationOfCamera2"])
    cameraMatrix1[0:2, 2] += 1
    cameraMatrix2[0:2, 2] += 1
    img = cv2.imread(ImagePath)
    img_undistorted = cv2.undistort(img, cameraMatrix1, distCoeffs1)
    filename = view + '_undistortedV2.jpg'
    cv2.imwrite(filename, img_undistorted)
    
Undistorter(ImagePath1,view1,CameraDataFile)
Undistorter(ImagePath2,view2,CameraDataFile)

In [7]:
LeftImagePath = 'left_undistorted.jpg'
RightImagePath = 'right_undistorted.jpg'

In [8]:
def EpipolarGeometry(CameraDataFile,LeftImagePath,RightImagePath):
    
    dd = loadmat(CameraDataFile)
    distCoeffs1 = deepcopy(dd["distortionCoefficients1"])
    distCoeffs2 = deepcopy(dd["distortionCoefficients2"])
    cameraMatrix1 = deepcopy(dd["intrinsicMatrix1"])
    cameraMatrix2 = deepcopy(dd["intrinsicMatrix2"])
    R = deepcopy(dd["rotationOfCamera2"])
    T = deepcopy(dd["translationOfCamera2"])
    cameraMatrix1[0:2, 2] += 1
    cameraMatrix2[0:2, 2] += 1
    # Projection matrices
    P1 = np.dot(cameraMatrix1, np.hstack((np.eye(3), np.zeros((3, 1)))))
    P2 = np.dot(cameraMatrix2, np.hstack((R, T.reshape(3, 1))))
    
    # Load the undistorted stereo images
    img_left = cv2.imread(LeftImagePath, cv2.IMREAD_GRAYSCALE)
    img_right = cv2.imread(RightImagePath, cv2.IMREAD_GRAYSCALE)

    # Initialize SIFT detector
    sift = cv2.SIFT_create()

    # Detect keypoints and compute descriptors
    keypoints_left, descriptors_left = sift.detectAndCompute(img_left, None)
    keypoints_right, descriptors_right = sift.detectAndCompute(img_right, None)

    # Undistort keypoints
    keypoints_left_undistorted = cv2.undistortPoints(np.array([kp.pt for kp in keypoints_left]).reshape(-1, 1, 2), cameraMatrix1, distCoeffs1, P=cameraMatrix1)
    keypoints_right_undistorted = cv2.undistortPoints(np.array([kp.pt for kp in keypoints_right]).reshape(-1, 1, 2), cameraMatrix2, distCoeffs2, P=cameraMatrix2)

    # Convert keypoints back to OpenCV KeyPoint format
    keypoints_left_undistorted = [cv2.KeyPoint(x[0][0], x[0][1], 1) for x in keypoints_left_undistorted]
    keypoints_right_undistorted = [cv2.KeyPoint(x[0][0], x[0][1], 1) for x in keypoints_right_undistorted]

    # Create FLANN matcher object
    FLANN_INDEX_KDTREE = 0
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)
    flann = cv2.FlannBasedMatcher(index_params, search_params)

    # Match descriptors of left and right images
    matches = flann.knnMatch(descriptors_left, descriptors_right, k=2)

    # Apply ratio test
    good_matches = []
    pts_left = []
    pts_right = []

    for m, n in matches:
        if m.distance < 0.8 * n.distance:
            good_matches.append(m)
            pts_left.append(keypoints_left_undistorted[m.queryIdx].pt)
            pts_right.append(keypoints_right_undistorted[m.trainIdx].pt)

    pts_left = np.int32(pts_left) 
    pts_right = np.int32(pts_right) 
    F, mask = cv2.findFundamentalMat(pts_left, 
                                     pts_right, 
                                     cv2.FM_LMEDS) 

    # We select only inlier points 
    pts_left = pts_left[mask.ravel() == 1] 
    pts_right = pts_right[mask.ravel() == 1] 


    return (F,keypoints_left_undistorted,keypoints_right_undistorted)

In [9]:
Epilor_Details = EpipolarGeometry(CameraDataFile,LeftImagePath,RightImagePath)
# print(Epilor_Details)

In [10]:
def find_matching_points(left_point, fundamental_matrix, right_image_keypoints):
    
    # Convert left point to homogeneous coordinates
    left_point = np.array([left_point[0], left_point[1], 1])

    # Compute the epipolar line in the right image corresponding to the left point
    epipolar_line = np.dot(fundamental_matrix, left_point)
    # Calculate the bounds for the search area in the right image
    search_area_width = 100  # You can adjust this based on your requirement
    search_area_height = 100  # You can adjust this based on your requirement

    # Define the search area
    search_area = ((left_point[0] - search_area_width, left_point[1] - search_area_height),
                   (left_point[0] + search_area_width, left_point[1] + search_area_height))

    # Initialize a list to store potential matching points
    potential_matching_points = []

    # Iterate through all keypoints in the right image
    for keypoint in right_image_keypoints:
        x, y = keypoint.pt

        # Check if the keypoint lies within the search area
        if search_area[0][0] <= x <= search_area[1][0] and search_area[0][1] <= y <= search_area[1][1]:
            # Compute the distance between the keypoint and the epipolar line
            distance = abs(epipolar_line[0] * x + epipolar_line[1] * y + epipolar_line[2]) / \
                       np.sqrt(epipolar_line[0] ** 2 + epipolar_line[1] ** 2)

            # You can adjust this threshold based on your requirement
            if distance < 30:  # Adjust this threshold as needed
                potential_matching_points.append((x, y))

    return potential_matching_points

In [11]:
def find_matching_points_reverse(right_point, fundamental_matrix, left_image_keypoints):
    
    # Convert left point to homogeneous coordinates
    right_point = np.array([right_point[0], right_point[1], 1])

    # Compute the epipolar line in the right image corresponding to the left point
    epipolar_line = np.dot(fundamental_matrix, right_point)
    # Calculate the bounds for the search area in the right image
    search_area_width = 100  # You can adjust this based on your requirement
    search_area_height = 100  # You can adjust this based on your requirement

    # Define the search area
    search_area = ((right_point[0] - search_area_width, right_point[1] - search_area_height),
                   (right_point[0] + search_area_width, right_point[1] + search_area_height))

    # Initialize a list to store potential matching points
    potential_matching_points = []

    # Iterate through all keypoints in the right image
    for keypoint in left_image_keypoints:
        x, y = keypoint.pt

        # Check if the keypoint lies within the search area
        if search_area[0][0] <= x <= search_area[1][0] and search_area[0][1] <= y <= search_area[1][1]:
            # Compute the distance between the keypoint and the epipolar line
            distance = abs(epipolar_line[0] * x + epipolar_line[1] * y + epipolar_line[2]) / \
                       np.sqrt(epipolar_line[0] ** 2 + epipolar_line[1] ** 2)

            # You can adjust this threshold based on your requirement
            if distance < 30:  # Adjust this threshold as needed
                potential_matching_points.append((x, y))

    return potential_matching_points

In [12]:
def closest_tuple(given_tuple, tuple_list):
    
    closest = None
    min_distance = float('inf')
    
    for tpl in tuple_list:
        distance = math.sqrt(sum((x - y) ** 2 for x, y in zip(given_tuple, tpl)))
        if distance < min_distance:
            min_distance = distance
            closest = tpl
    
    return closest

In [13]:
def tuple_differences(tuple1, tuple2):
    if len(tuple1) != len(tuple2):
        raise ValueError("Tuples must have the same length")
    
    return tuple(x - y for x, y in zip(tuple1, tuple2))

In [14]:
def Find_Matching_ID(left_id, left_point_tuple, right_view_dataframe, fundamental_matrix, right_image_keypoints):
    
    matching_points = find_matching_points(left_point_tuple, fundamental_matrix, right_image_keypoints)
    
    options = []
    
    df = pd.DataFrame(columns=['left_id','right_id','difference','frame'])
    difference = None
    for index, row in right_view_dataframe.iterrows():
        tuple_right = (row['x'],row['y'])
        
        difference = abs(tuple_differences(tuple_right,closest_tuple(tuple_right, matching_points))[0]) + abs(tuple_differences(tuple_right,closest_tuple(tuple_right, matching_points))[1])
        new_row_data = {'left_id': left_id, 'right_id': row['id'],'difference': difference,'frame':row['frame']}
        new_row = pd.DataFrame(new_row_data,index=[0])
        df = pd.concat([df, new_row], ignore_index=True)

        
    return df

In [15]:
def Find_Matching_ID_right_to_left(right_id, right_point_tuple, left_view_dataframe, fundamental_matrix, left_image_keypoints):
    
    matching_points = find_matching_points_reverse(right_point_tuple, fundamental_matrix, left_image_keypoints)
    
    options = []
    
    df = pd.DataFrame(columns=['right_id','left_id','difference','frame'])
    difference = None
    for index, row in left_view_dataframe.iterrows():
        tuple_left = (row['x'],row['y'])
        
        difference = abs(tuple_differences(tuple_left,closest_tuple(tuple_left, matching_points))[0]) + abs(tuple_differences(tuple_left,closest_tuple(tuple_left, matching_points))[1])
        new_row_data = {'right_id': right_id, 'left_id': row['id'],'difference': difference,'frame':row['frame']}
        new_row = pd.DataFrame(new_row_data,index=[0])
        df = pd.concat([df, new_row], ignore_index=True)

        
    return df

In [17]:
fundamental_matrix = Epilor_Details[0]
left_id = 6
left_point_tuple = (1287.82,366.35)
right_view_dataframe = df2_right_view
right_image_keypoints = Epilor_Details[2]
left_image_keypoints = Epilor_Details[1]

In [20]:
# print("Most common right_id:", most_common_id)
frames_to_check = [796,797,798, 799, 800,801,802,803]  # Example list of frames to check

# Filter df2_right_view for frames in frames_to_check
# df_to_use_test_reverse = df2_right_view[df2_right_view['frame'].isin(frames_to_check)]
df_to_use_test_reverse = df2_right_view

# Display the resulting DataFrame
display(df_to_use_test_reverse)

# df_to_check_test_reverse = df1_left_view[df1_left_view['frame'].isin(frames_to_check)]
df_to_check_test_reverse = df1_left_view

# Display the resulting DataFrame
display(df_to_check_test_reverse)

set_frames_df2_right = set(df2_right_view['frame'])

,frame,id,x,y,x_offset,y_offset
0,1,1,1010.39,357.55,21.50,7.74
1,1,2,1151.46,337.71,20.11,17.63
2,1,3,1173.87,341.47,11.20,12.02
3,1,4,1446.30,460.60,19.11,20.90
4,1,5,1066.87,626.29,48.07,6.87
...,...,...,...,...,...,...
30644,3118,5,949.07,696.08,23.25,33.30
30645,3118,6,1067.44,288.15,38.96,21.85
30646,3118,7,1188.06,193.33,33.94,17.14
30647,3118,8,883.91,514.11,25.29,14.69


,frame,id,x,y,x_offset,y_offset
0,1,1,1212.98,416.74,26.04,14.37
1,1,2,1151.14,415.00,27.58,10.34
2,1,3,1487.32,431.94,17.38,19.86
3,1,4,1080.11,337.38,18.68,6.46
4,1,5,1220.90,325.30,21.28,10.54
...,...,...,...,...,...,...
30196,3117,5,1011.60,235.99,10.38,22.01
30197,3117,6,1304.99,217.40,15.01,24.80
30198,3117,7,969.60,489.40,23.90,16.40
30199,3117,8,1037.36,245.93,14.89,29.79


In [21]:
print(len(set_frames_df2_right))
print(len(set_frames_df1_left))
overlapping_frames = set_frames_df1_left & set_frames_df2_right
print(len(overlapping_frames))



# List of allowed values for 'right_id'
allowed_values = overlapping_frames

# Filter the DataFrame based on the condition

df_to_use_test = df_to_use_test[df_to_use_test['frame'].isin(allowed_values)]

df_to_check_test = df_to_check_test[df_to_check_test['frame'].isin(allowed_values)]

df_to_use_test_reverse = df_to_use_test_reverse[df_to_use_test_reverse['frame'].isin(allowed_values)]

df_to_check_test_reverse = df_to_check_test_reverse[df_to_check_test_reverse['frame'].isin(allowed_values)]

print(len(df_to_use_test))
print(len(df_to_check_test))
print(len(df_to_use_test_reverse))
print(len(df_to_check_test_reverse))

3118
3117
3117
30201
30636
30636
30201


In [22]:
def differences_per_frame_left_to_right(df_to_use_test, df_to_check_test,fundamental_matrix,right_image_keypoints):
    df_preds = pd.DataFrame()
    frames = set(df_to_use_test['frame'])
    for frame in frames:
        df_to_use = df_to_use_test[df_to_use_test['frame'] == frame]
        df_to_check = df_to_check_test[df_to_check_test['frame'] == frame]
    #     display(df_to_use)

        for index, row in df_to_use.iterrows():
            frame = row['frame']
            id_value = row['id']
            x = row['x']
            y = row['y']
            left_id = id_value
            left_point_tuple = (x,y)
            df_mathces = Find_Matching_ID(left_id, left_point_tuple, df_to_check, fundamental_matrix, right_image_keypoints)
            min_difference_index = df_mathces['difference'].idxmin()

            # Retrieve the corresponding row from the DataFrame
            row_with_smallest_difference = df_mathces.loc[min_difference_index]

            # Convert the row to a DataFrame
            row_df = row_with_smallest_difference.to_frame().T

            df_preds = pd.concat([df_preds,row_df],ignore_index=True)

    return(df_preds)

In [23]:
df_preds = differences_per_frame_left_to_right(df_to_use_test,df_to_check_test,fundamental_matrix,right_image_keypoints)
df_preds

,left_id,right_id,difference,frame
0,1.0,6.0,27.999137,1.0
1,2.0,6.0,4.070687,1.0
2,3.0,4.0,2.869824,1.0
3,4.0,1.0,4.092777,1.0
4,5.0,3.0,6.089725,1.0
...,...,...,...,...
30196,5.0,11.0,3.773182,3117.0
30197,6.0,2.0,12.651285,3117.0
30198,7.0,8.0,10.728657,3117.0
30199,8.0,11.0,3.773182,3117.0


In [24]:
def differences_per_frame_right_to_left(df_to_use_test_reverse, df_to_check_test_reverse,fundamental_matrix,left_image_keypoints):
    df_preds = pd.DataFrame()
    frames = set(df_to_use_test_reverse['frame'])
    for frame in frames:
        df_to_use = df_to_use_test_reverse[df_to_use_test_reverse['frame'] == frame]
        df_to_check = df_to_check_test_reverse[df_to_check_test_reverse['frame'] == frame]
    #     display(df_to_use)

        for index, row in df_to_use.iterrows():
            frame = row['frame']
            id_value = row['id']
            x = row['x']
            y = row['y']
            right_id = id_value
            right_point_tuple = (x,y)
            df_mathces = Find_Matching_ID_right_to_left(right_id, right_point_tuple, df_to_check, fundamental_matrix, left_image_keypoints)
#             print(df_mathces)
            min_difference_index = df_mathces['difference'].idxmin()
#             print("sucess")

            # Retrieve the corresponding row from the DataFrame
            row_with_smallest_difference = df_mathces.loc[min_difference_index]

            # Convert the row to a DataFrame
            row_df = row_with_smallest_difference.to_frame().T

            df_preds = pd.concat([df_preds,row_df],ignore_index=True)

    return(df_preds)

In [25]:
df_preds_reverse = differences_per_frame_right_to_left(df_to_use_test_reverse,df_to_check_test_reverse,fundamental_matrix,left_image_keypoints)

df_preds_reverse




,right_id,left_id,difference,frame
0,1.0,4.0,5.101002,1.0
1,2.0,4.0,5.101002,1.0
2,3.0,4.0,5.101002,1.0
3,4.0,3.0,30.350840,1.0
4,5.0,9.0,59.642593,1.0
...,...,...,...,...
30631,5.0,9.0,87.986079,3117.0
30632,6.0,11.0,30.152407,3117.0
30633,7.0,2.0,6.788995,3117.0
30634,8.0,7.0,34.664368,3117.0


In [27]:
def get_matches_left_to_right(df_preds):
    # Group by frame and left_id, find the right_id with the lowest difference
    min_diff_indices = df_preds.groupby(['frame', 'left_id'])['difference'].idxmin()

    # Get the corresponding right_id values with the lowest difference
    matches = df_preds.loc[min_diff_indices][['left_id', 'right_id']].groupby('left_id')['right_id'].apply(list).reset_index()

    # Rename the column for clarity
    matches.rename(columns={'right_id': 'matches'}, inplace=True)

    return matches
    

In [28]:
matches = get_matches_left_to_right(df_preds)
display(matches)

,left_id,matches
0,1.0,"[6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, ..."
1,2.0,"[6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, ..."
2,3.0,"[4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, ..."
3,4.0,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 3.0, 3.0, 3.0, ..."
4,5.0,"[3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, ..."
5,6.0,"[3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, ..."
6,7.0,"[9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, ..."
7,8.0,"[8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, ..."
8,9.0,"[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ..."
9,10.0,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."


In [29]:
def get_matches_right_to_left(df_preds_reverse):
    # Group by frame and left_id, find the right_id with the lowest difference
    min_diff_indices = df_preds_reverse.groupby(['frame', 'right_id'])['difference'].idxmin()

    # Get the corresponding right_id values with the lowest difference
    matches = df_preds_reverse.loc[min_diff_indices][['right_id', 'left_id']].groupby('right_id')['left_id'].apply(list).reset_index()

    # Rename the column for clarity
    matches.rename(columns={'left_id': 'matches'}, inplace=True)

    return matches
    

In [30]:
matches_reverse = get_matches_right_to_left(df_preds_reverse)
display(matches_reverse)

,right_id,matches
0,1.0,"[4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, ..."
1,2.0,"[4.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ..."
2,3.0,"[4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, ..."
3,4.0,"[3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, ..."
4,5.0,"[9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, ..."
5,6.0,"[8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, ..."
6,7.0,"[8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, ..."
7,8.0,"[7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, ..."
8,9.0,"[7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, ..."
9,10.0,"[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10...."


In [32]:
from collections import Counter
def top_3_common(lst):
    counts = {}
    for item in lst:
        counts[item] = counts.get(item, 0) + 1
    sorted_counts = sorted(counts.items(), key=lambda x: x[1], reverse=True)
    return [item[0] for item in sorted_counts[:3]]

def top_three_left_to_right(matches):
    # Apply the function to each row and replace the 'matches' values
    matches['matches'] = matches['matches'].apply(top_3_common)
    matches.rename(columns={'matches': 'top_Three_Right_ids'}, inplace=True)
    return matches


def top_three_right_to_left(matches_reverse):
    # Apply the function to each row and replace the 'matches' values
    matches_reverse['matches'] = matches_reverse['matches'].apply(top_3_common)
    matches_reverse.rename(columns={'matches': 'top_Three_left_ids'}, inplace=True)
    return matches_reverse

In [33]:
top_three_left_to_right = top_three_left_to_right(matches)
display(top_three_left_to_right)
top_three_right_to_left = top_three_right_to_left(matches_reverse)
display(top_three_right_to_left)

,left_id,top_Three_Right_ids
0,1.0,"[6.0, 8.0, 2.0]"
1,2.0,"[7.0, 3.0, 6.0]"
2,3.0,[4.0]
3,4.0,"[1.0, 12.0, 3.0]"
4,5.0,"[3.0, 7.0, 11.0]"
5,6.0,"[2.0, 3.0, 7.0]"
6,7.0,"[8.0, 6.0, 9.0]"
7,8.0,"[10.0, 7.0, 9.0]"
8,9.0,"[5.0, 8.0, 6.0]"
9,10.0,"[10.0, 11.0, 1.0]"


,right_id,top_Three_left_ids
0,1.0,"[4.0, 8.0, 11.0]"
1,2.0,"[6.0, 1.0, 12.0]"
2,3.0,"[5.0, 6.0, 8.0]"
3,4.0,"[3.0, 1.0]"
4,5.0,"[9.0, 7.0, 1.0]"
5,6.0,"[7.0, 9.0, 1.0]"
6,7.0,"[2.0, 5.0, 8.0]"
7,8.0,"[7.0, 9.0, 1.0]"
8,9.0,"[8.0, 10.0, 4.0]"
9,10.0,"[10.0, 8.0, 11.0]"


In [34]:
df1 = top_three_left_to_right
df2 = top_three_right_to_left

d_lr = []

for i in range(len(df1)):
    left_id = df1.iloc[i]['left_id']
    right_ids = df1.iloc[i]['top_Three_Right_ids']
    for ID in right_ids:
        pot = list(df2['top_Three_left_ids'].loc[df2['right_id'] == ID])
        pot = pot[0]
#         print(pot,ID,left_id,right_ids)
        if left_id in pot:
#             print(left_id,ID,"MATCH",pot,right_ids)
            d_lr.append((left_id,ID))
        
print(d_lr)

[(1.0, 6.0), (1.0, 8.0), (1.0, 2.0), (2.0, 7.0), (3.0, 4.0), (4.0, 1.0), (4.0, 12.0), (5.0, 3.0), (5.0, 7.0), (6.0, 2.0), (6.0, 3.0), (7.0, 8.0), (7.0, 6.0), (8.0, 10.0), (8.0, 7.0), (8.0, 9.0), (9.0, 5.0), (9.0, 8.0), (9.0, 6.0), (10.0, 10.0), (11.0, 11.0), (11.0, 1.0), (11.0, 12.0), (12.0, 12.0), (12.0, 2.0), (13.0, 13.0), (14.0, 14.0), (15.0, 15.0)]


In [35]:
d_rl = []

for i in range(len(df2)):
    right_id = df2.iloc[i]['right_id']
    left_ids = df2.iloc[i]['top_Three_left_ids']
    for ID in left_ids:
        pot = list(df1['top_Three_Right_ids'].loc[df1['left_id'] == ID])
        pot = pot[0]
#         print(pot,ID,left_id,right_ids)
        if right_id in pot:
#             print(left_id,ID,"MATCH")
            d_rl.append((right_id,ID))
        
print(d_rl)

[(1.0, 4.0), (1.0, 11.0), (2.0, 6.0), (2.0, 1.0), (2.0, 12.0), (3.0, 5.0), (3.0, 6.0), (4.0, 3.0), (5.0, 9.0), (6.0, 7.0), (6.0, 9.0), (6.0, 1.0), (7.0, 2.0), (7.0, 5.0), (7.0, 8.0), (8.0, 7.0), (8.0, 9.0), (8.0, 1.0), (9.0, 8.0), (10.0, 10.0), (10.0, 8.0), (11.0, 11.0), (12.0, 4.0), (12.0, 11.0), (12.0, 12.0), (13.0, 13.0), (14.0, 14.0), (15.0, 15.0)]


In [36]:
left_list = []
right_list = []
pos_match = pd.DataFrame()
for tup in d_lr:
    l = tup[0]
    r = tup[1]
    if (r,l) in d_rl:
#         print(tup,(r,l))
        left_list.append(l)
        right_list.append(r)
        df = pd.DataFrame({'left_right': [tup]})
#         print(df)
        pos_match = pd.concat([pos_match, df], ignore_index=True)
        

pos_match['left_boy'] = left_list
pos_match['right_boy'] = right_list
pos_match

,left_right,left_boy,right_boy
0,"(1.0, 6.0)",1.0,6.0
1,"(1.0, 8.0)",1.0,8.0
2,"(1.0, 2.0)",1.0,2.0
3,"(2.0, 7.0)",2.0,7.0
4,"(3.0, 4.0)",3.0,4.0
5,"(4.0, 1.0)",4.0,1.0
6,"(4.0, 12.0)",4.0,12.0
7,"(5.0, 3.0)",5.0,3.0
8,"(5.0, 7.0)",5.0,7.0
9,"(6.0, 2.0)",6.0,2.0


In [37]:
for x,y in pos_match['left_right']:
    print(x,y)

1.0 6.0
1.0 8.0
1.0 2.0
2.0 7.0
3.0 4.0
4.0 1.0
4.0 12.0
5.0 3.0
5.0 7.0
6.0 2.0
6.0 3.0
7.0 8.0
7.0 6.0
8.0 10.0
8.0 7.0
8.0 9.0
9.0 5.0
9.0 8.0
9.0 6.0
10.0 10.0
11.0 11.0
11.0 1.0
11.0 12.0
12.0 12.0
12.0 2.0
13.0 13.0
14.0 14.0
15.0 15.0


In [38]:
df_preds

,left_id,right_id,difference,frame
0,1.0,6.0,27.999137,1.0
1,2.0,6.0,4.070687,1.0
2,3.0,4.0,2.869824,1.0
3,4.0,1.0,4.092777,1.0
4,5.0,3.0,6.089725,1.0
...,...,...,...,...
30196,5.0,11.0,3.773182,3117.0
30197,6.0,2.0,12.651285,3117.0
30198,7.0,8.0,10.728657,3117.0
30199,8.0,11.0,3.773182,3117.0


In [39]:
# Assuming your dataframe is named df
average_difference = df_preds[(df_preds['left_id'] == 1) & (df_preds['right_id'] == 6)]['difference'].mean()

print("Average difference:", average_difference)

Average difference: 31.147638694003643


In [40]:
averageDifferences_left_to_right = []
for left_id,right_id in pos_match['left_right']:
    average_difference = df_preds[(df_preds['left_id'] == left_id) & (df_preds['right_id'] == right_id)]['difference'].mean()
    print(left_id,right_id,average_difference)
    averageDifferences_left_to_right.append(average_difference)
    
averageDifferences_left_to_right
    

1.0 6.0 31.147638694003643
1.0 8.0 31.03824310625827
1.0 2.0 31.512393012612563
2.0 7.0 28.305854631512073
3.0 4.0 3.5205847361117923
4.0 1.0 5.129555660625512
4.0 12.0 2.568190511067709
5.0 3.0 6.581941293532817
5.0 7.0 5.732718550159082
6.0 2.0 6.350316482447846
6.0 3.0 6.375270315616695
7.0 8.0 9.355854424243995
7.0 6.0 3.973227684384299
8.0 10.0 7.426428780087156
8.0 7.0 12.35015355299526
8.0 9.0 25.867069772992824
9.0 5.0 46.498770953119745
9.0 8.0 45.92902578484156
9.0 6.0 13.537368150346735
10.0 10.0 6.784330416107961
11.0 11.0 4.638806731908964
11.0 1.0 2.411324707031246
11.0 12.0 4.043863319074605
12.0 12.0 2.9893510800003407
12.0 2.0 1.745077049126047
13.0 13.0 4.173788195584727
14.0 14.0 2.936132365070977
15.0 15.0 12.15722656250011


[31.147638694003643,
 31.03824310625827,
 31.512393012612563,
 28.305854631512073,
 3.5205847361117923,
 5.129555660625512,
 2.568190511067709,
 6.581941293532817,
 5.732718550159082,
 6.350316482447846,
 6.375270315616695,
 9.355854424243995,
 3.973227684384299,
 7.426428780087156,
 12.35015355299526,
 25.867069772992824,
 46.498770953119745,
 45.92902578484156,
 13.537368150346735,
 6.784330416107961,
 4.638806731908964,
 2.411324707031246,
 4.043863319074605,
 2.9893510800003407,
 1.745077049126047,
 4.173788195584727,
 2.936132365070977,
 12.15722656250011]

In [41]:
averageDifferences_right_to_left = []
for left_id,right_id in pos_match['left_right']:
    average_difference = df_preds_reverse[(df_preds_reverse['right_id'] == right_id) & (df_preds_reverse['left_id'] == left_id)]['difference'].mean()
    print(right_id,left_id,average_difference)
    averageDifferences_right_to_left.append(average_difference)
    
averageDifferences_right_to_left
    

6.0 1.0 46.873185486249476
8.0 1.0 8.173759375577843
2.0 1.0 9.001780522078406
7.0 2.0 26.74649264958504
4.0 3.0 35.46459374536442
1.0 4.0 8.657923758801113
12.0 4.0 4.52345605872765
3.0 5.0 8.135520986588714
7.0 5.0 16.581629420900807
2.0 6.0 16.584583328467932
3.0 6.0 3.8249790584679855
8.0 7.0 20.249184003328022
6.0 7.0 21.318115187978684
10.0 8.0 8.467741246541339
7.0 8.0 9.076426234962666
9.0 8.0 52.99960883514497
5.0 9.0 69.48484980573612
8.0 9.0 14.123383041943194
6.0 9.0 38.31299602693914
10.0 10.0 9.778600711633668
11.0 11.0 12.07148373773915
1.0 11.0 3.6579113624783846
12.0 11.0 8.520924812376967
12.0 12.0 7.686644628131199
2.0 12.0 8.909782054304129
13.0 13.0 8.24131825193484
14.0 14.0 49.261029602294
15.0 15.0 54.20410852502884


[46.873185486249476,
 8.173759375577843,
 9.001780522078406,
 26.74649264958504,
 35.46459374536442,
 8.657923758801113,
 4.52345605872765,
 8.135520986588714,
 16.581629420900807,
 16.584583328467932,
 3.8249790584679855,
 20.249184003328022,
 21.318115187978684,
 8.467741246541339,
 9.076426234962666,
 52.99960883514497,
 69.48484980573612,
 14.123383041943194,
 38.31299602693914,
 9.778600711633668,
 12.07148373773915,
 3.6579113624783846,
 8.520924812376967,
 7.686644628131199,
 8.909782054304129,
 8.24131825193484,
 49.261029602294,
 54.20410852502884]

In [42]:
avg_diff = [diflr + difrl for diflr, difrl in zip(averageDifferences_left_to_right, averageDifferences_right_to_left)]
pos_match['avg_diff'] = avg_diff
pos_match['left_right_diff'] = averageDifferences_left_to_right
pos_match['right_left_diff'] = averageDifferences_right_to_left

In [43]:
pos_match

,left_right,left_boy,right_boy,avg_diff,left_right_diff,right_left_diff
0,"(1.0, 6.0)",1.0,6.0,78.020824,31.147639,46.873185
1,"(1.0, 8.0)",1.0,8.0,39.212002,31.038243,8.173759
2,"(1.0, 2.0)",1.0,2.0,40.514174,31.512393,9.001781
3,"(2.0, 7.0)",2.0,7.0,55.052347,28.305855,26.746493
4,"(3.0, 4.0)",3.0,4.0,38.985178,3.520585,35.464594
5,"(4.0, 1.0)",4.0,1.0,13.787479,5.129556,8.657924
6,"(4.0, 12.0)",4.0,12.0,7.091647,2.568191,4.523456
7,"(5.0, 3.0)",5.0,3.0,14.717462,6.581941,8.135521
8,"(5.0, 7.0)",5.0,7.0,22.314348,5.732719,16.581629
9,"(6.0, 2.0)",6.0,2.0,22.934900,6.350316,16.584583


In [44]:
# Convert tuples to string for easy grouping
df = pos_match

# Convert tuples to string for easy grouping
df['left_right'] = df['left_right'].astype(str)

# Group by the first element of the tuple and select the row with the minimum avg_diff
result_df = df.loc[df.groupby(df['left_right'].str.split(',').str[0])['avg_diff'].idxmin()]

# Reset index
result_df.reset_index(drop=True, inplace=True)

display(result_df)

,left_right,left_boy,right_boy,avg_diff,left_right_diff,right_left_diff
0,"(1.0, 8.0)",1.0,8.0,39.212002,31.038243,8.173759
1,"(10.0, 10.0)",10.0,10.0,16.562931,6.784330,9.778601
2,"(11.0, 1.0)",11.0,1.0,6.069236,2.411325,3.657911
3,"(12.0, 2.0)",12.0,2.0,10.654859,1.745077,8.909782
4,"(13.0, 13.0)",13.0,13.0,12.415106,4.173788,8.241318
5,"(14.0, 14.0)",14.0,14.0,52.197162,2.936132,49.261030
6,"(15.0, 15.0)",15.0,15.0,66.361335,12.157227,54.204109
7,"(2.0, 7.0)",2.0,7.0,55.052347,28.305855,26.746493
8,"(3.0, 4.0)",3.0,4.0,38.985178,3.520585,35.464594
9,"(4.0, 12.0)",4.0,12.0,7.091647,2.568191,4.523456


In [45]:
df_preds.loc[df_preds['frame'] == 318]

,left_id,right_id,difference,frame
2852,1.0,6.0,21.437627,318.0
2853,2.0,6.0,11.577017,318.0
2854,4.0,1.0,4.167118,318.0
2855,5.0,3.0,1.597996,318.0
2856,6.0,2.0,1.191216,318.0
2857,7.0,6.0,0.504888,318.0
2858,8.0,6.0,0.504888,318.0
2859,9.0,5.0,53.880613,318.0


In [46]:
df_preds_reverse.loc[df_preds_reverse['frame'] == 318]

,right_id,left_id,difference,frame
2853,1.0,4.0,13.838596,318.0
2854,2.0,6.0,9.890081,318.0
2855,3.0,6.0,5.639343,318.0
2856,4.0,1.0,215.552112,318.0
2857,5.0,9.0,72.734407,318.0
2858,6.0,7.0,23.257141,318.0
2859,7.0,8.0,0.342428,318.0
2860,8.0,7.0,23.257141,318.0
2861,9.0,8.0,67.318918,318.0


Not every frame has the same amount of fish in it which causes matching errors when taking the best matches of all frames together to determine to best matches, due to this the best approach is to take the best match per frame each time. Matches change per frame when looking at the data by hand. 

In [47]:
# Subtract 1 from each value in 'left_id' and 'right_id'
df_preds['right_id'] = df_preds['right_id'].apply(lambda x: x - 1)
df_preds['left_id'] = df_preds['left_id'].apply(lambda x: x - 1)
df_preds

,left_id,right_id,difference,frame
0,0.0,5.0,27.999137,1.0
1,1.0,5.0,4.070687,1.0
2,2.0,3.0,2.869824,1.0
3,3.0,0.0,4.092777,1.0
4,4.0,2.0,6.089725,1.0
...,...,...,...,...
30196,4.0,10.0,3.773182,3117.0
30197,5.0,1.0,12.651285,3117.0
30198,6.0,7.0,10.728657,3117.0
30199,7.0,10.0,3.773182,3117.0


In [50]:
pos_match

#     left_list.append(pos_match['left_right'][x][])

,left_right,left_boy,right_boy,avg_diff,left_right_diff,right_left_diff
0,"(1.0, 6.0)",1.0,6.0,78.020824,31.147639,46.873185
1,"(1.0, 8.0)",1.0,8.0,39.212002,31.038243,8.173759
2,"(1.0, 2.0)",1.0,2.0,40.514174,31.512393,9.001781
3,"(2.0, 7.0)",2.0,7.0,55.052347,28.305855,26.746493
4,"(3.0, 4.0)",3.0,4.0,38.985178,3.520585,35.464594
5,"(4.0, 1.0)",4.0,1.0,13.787479,5.129556,8.657924
6,"(4.0, 12.0)",4.0,12.0,7.091647,2.568191,4.523456
7,"(5.0, 3.0)",5.0,3.0,14.717462,6.581941,8.135521
8,"(5.0, 7.0)",5.0,7.0,22.314348,5.732719,16.581629
9,"(6.0, 2.0)",6.0,2.0,22.934900,6.350316,16.584583


In [51]:
pos_match['right_boy'] = pos_match['right_boy'].apply(lambda x: x - 1)
pos_match['left_boy'] = pos_match['left_boy'].apply(lambda x: x - 1)
pos_match

,left_right,left_boy,right_boy,avg_diff,left_right_diff,right_left_diff
0,"(1.0, 6.0)",0.0,5.0,78.020824,31.147639,46.873185
1,"(1.0, 8.0)",0.0,7.0,39.212002,31.038243,8.173759
2,"(1.0, 2.0)",0.0,1.0,40.514174,31.512393,9.001781
3,"(2.0, 7.0)",1.0,6.0,55.052347,28.305855,26.746493
4,"(3.0, 4.0)",2.0,3.0,38.985178,3.520585,35.464594
5,"(4.0, 1.0)",3.0,0.0,13.787479,5.129556,8.657924
6,"(4.0, 12.0)",3.0,11.0,7.091647,2.568191,4.523456
7,"(5.0, 3.0)",4.0,2.0,14.717462,6.581941,8.135521
8,"(5.0, 7.0)",4.0,6.0,22.314348,5.732719,16.581629
9,"(6.0, 2.0)",5.0,1.0,22.934900,6.350316,16.584583


In [49]:
result_df

,left_right,left_boy,right_boy,avg_diff,left_right_diff,right_left_diff
0,"(1.0, 8.0)",1.0,8.0,39.212002,31.038243,8.173759
1,"(10.0, 10.0)",10.0,10.0,16.562931,6.784330,9.778601
2,"(11.0, 1.0)",11.0,1.0,6.069236,2.411325,3.657911
3,"(12.0, 2.0)",12.0,2.0,10.654859,1.745077,8.909782
4,"(13.0, 13.0)",13.0,13.0,12.415106,4.173788,8.241318
5,"(14.0, 14.0)",14.0,14.0,52.197162,2.936132,49.261030
6,"(15.0, 15.0)",15.0,15.0,66.361335,12.157227,54.204109
7,"(2.0, 7.0)",2.0,7.0,55.052347,28.305855,26.746493
8,"(3.0, 4.0)",3.0,4.0,38.985178,3.520585,35.464594
9,"(4.0, 12.0)",4.0,12.0,7.091647,2.568191,4.523456


In [52]:
result_df['right_boy'] = result_df['right_boy'].apply(lambda x: x - 1)
result_df['left_boy'] = result_df['left_boy'].apply(lambda x: x - 1)
result_df

,left_right,left_boy,right_boy,avg_diff,left_right_diff,right_left_diff
0,"(1.0, 8.0)",0.0,7.0,39.212002,31.038243,8.173759
1,"(10.0, 10.0)",9.0,9.0,16.562931,6.784330,9.778601
2,"(11.0, 1.0)",10.0,0.0,6.069236,2.411325,3.657911
3,"(12.0, 2.0)",11.0,1.0,10.654859,1.745077,8.909782
4,"(13.0, 13.0)",12.0,12.0,12.415106,4.173788,8.241318
5,"(14.0, 14.0)",13.0,13.0,52.197162,2.936132,49.261030
6,"(15.0, 15.0)",14.0,14.0,66.361335,12.157227,54.204109
7,"(2.0, 7.0)",1.0,6.0,55.052347,28.305855,26.746493
8,"(3.0, 4.0)",2.0,3.0,38.985178,3.520585,35.464594
9,"(4.0, 12.0)",3.0,11.0,7.091647,2.568191,4.523456


In [54]:
# Save the DataFrame as a CSV file
df_preds.to_csv('df_preds_v2.csv', index=False)

In [55]:
pos_match.to_csv('pos_match_v2.csv', index=False)
result_df.to_csv('result_df_v2.csv', index=False)